In [2]:
import pandas as pd
import numpy as np

In [3]:
#load January 2024 data
df_2024 = pd.read_csv('JC-202401-citibike-tripdata.csv')
print(len(df_2024))
df_2024.head()

50661


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,40.737604,-74.052478,member
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,40.719410,-74.043090,member
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,40.719252,-74.034234,member


In [4]:
# Create dfs for start and end stations w/ simplified column names

start_stations_df = df_2024[['start_station_name', 'start_station_id', 'start_lat', 'start_lng']].rename(
    columns={
        'start_station_name': 'station_name',
        'start_station_id': 'station_id',
        'start_lat': 'latitude',
        'start_lng': 'longitude'
    }
)

end_stations_df = df_2024[['end_station_name', 'end_station_id', 'end_lat', 'end_lng']].rename(
    columns={
        'end_station_name': 'station_name',
        'end_station_id': 'station_id',
        'end_lat': 'latitude',
        'end_lng': 'longitude'
    }
)

In [5]:
#check new dfs and length to make sure concatenation works correctly
print(len(start_stations_df))
start_stations_df.head()

50661


,station_name,station_id,latitude,longitude
0,Morris Canal,JC072,40.712297,-74.038185
1,JC Medical Center,JC110,40.715391,-74.049692
2,Morris Canal,JC072,40.712419,-74.038526
3,Morris Canal,JC072,40.712419,-74.038526
4,Morris Canal,JC072,40.712419,-74.038526


In [6]:
#check new dfs and length to make sure concatenation works correctly
print(len(end_stations_df))
end_stations_df.head()

50661


,station_name,station_id,latitude,longitude
0,Oakland Ave,JC022,40.737604,-74.052478
1,Grove St PATH,JC115,40.719410,-74.043090
2,Exchange Pl,JC116,40.716366,-74.034344
3,Exchange Pl,JC116,40.716366,-74.034344
4,Harborside,JC104,40.719252,-74.034234


In [7]:
#Concatenate the two dfs, reset the index
all_stations_df = pd.concat([start_stations_df, end_stations_df], ignore_index=True)
print(len(all_stations_df))
all_stations_df.head()

101322


,station_name,station_id,latitude,longitude
0,Morris Canal,JC072,40.712297,-74.038185
1,JC Medical Center,JC110,40.715391,-74.049692
2,Morris Canal,JC072,40.712419,-74.038526
3,Morris Canal,JC072,40.712419,-74.038526
4,Morris Canal,JC072,40.712419,-74.038526


In [8]:
#keep only one set of coordinates for each station and drop NaN 
if 'station_id' in all_stations_df.columns and all_stations_df['station_id'].notna().all():
    all_stations_df = all_stations_df.drop_duplicates(subset='station_id', keep='first')
    all_stations_df = all_stations_df.dropna(subset=['station_name', 'station_id', 'latitude', 'longitude'])
    all_stations_df = all_stations_df.drop_duplicates(subset=['station_name'], keep='first')
else:
    all_stations_df = all_stations_df.dropna(subset=['station_name', 'latitude', 'longitude'])
    all_stations_df = all_stations_df.drop_duplicates(subset=['station_name'], keep='first')

In [9]:
print(f'unique stations: {len(all_stations_df)}')

unique stations: 127


In [10]:
all_stations_df.head()

,station_name,station_id,latitude,longitude
0,Morris Canal,JC072,40.712297,-74.038185
1,JC Medical Center,JC110,40.715391,-74.049692
6,Church Sq Park - 5 St & Park Ave,HB601,40.742659,-74.032233
17,Riverview Park,JC057,40.744740,-74.043801
23,Jersey & 3rd,JC074,40.723332,-74.045953


In [11]:
#export csv
all_stations_df.to_csv('cleaned_stations.csv', index=False)

In [12]:
#drop all lat and lon from January data
dropped_2024_df = df_2024.drop(['start_lat', 'start_lng', 'end_lat', 'end_lng'], axis=1)

#merge with all_stations_df to add in corrected start lat and lng, then drop extra columns
almost_2024_df = pd.merge(dropped_2024_df, all_stations_df, left_on= 'start_station_id', right_on= 'station_id', how='left')
almost_2024_df = almost_2024_df.rename(columns={'latitude': 'start_lat', 'longitude': 'start_lng'}).drop(['station_name', 'station_id'], axis=1)

#merge with all_stations_df to add in corrected end lat and lng, then drop extra columns
clean_2024_df = pd.merge(almost_2024_df, all_stations_df, left_on= 'end_station_id', right_on= 'station_id', how='left')
clean_2024_df = clean_2024_df.rename(columns={'latitude': 'end_lat', 'longitude': 'end_lng'}).drop(['station_name', 'station_id'], axis=1)

print(len(clean_2024_df))
clean_2024_df.head()

50661


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,start_lat,start_lng,end_lat,end_lng
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,member,40.712297,-74.038185,40.737587,-74.052542
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,member,40.715391,-74.049692,40.719121,-74.043265
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,member,40.712297,-74.038185,40.716366,-74.034344
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,member,40.712297,-74.038185,40.716366,-74.034344
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,member,40.712297,-74.038185,40.719252,-74.034234


In [13]:
#export cleaned 2024 csv
clean_2024_df.to_csv('cleaned_2024.csv', index=False)

In [14]:
#do it all again for 2022
#load January 2022 data
df_2022 = pd.read_csv('JC-202201-citibike-tripdata.csv')

#drop all lat and lon from January data
dropped_2022_df = df_2022.drop(['start_lat', 'start_lng', 'end_lat', 'end_lng'], axis=1)

#merge with all_stations_df to add in corrected start lat and lng, then drop extra columns
almost_2022_df = pd.merge(dropped_2022_df, all_stations_df, left_on= 'start_station_id', right_on= 'station_id', how='left')
almost_2022_df = almost_2022_df.rename(columns={'latitude': 'start_lat', 'longitude': 'start_lng'}).drop(['station_name', 'station_id'], axis=1)

#merge with all_stations_df to add in corrected end lat and lng, then drop extra columns
clean_2022_df = pd.merge(almost_2022_df, all_stations_df, left_on= 'end_station_id', right_on= 'station_id', how='left')
clean_2022_df = clean_2022_df.rename(columns={'latitude': 'end_lat', 'longitude': 'end_lng'}).drop(['station_name', 'station_id'], axis=1)

#export cleaned 2022 csv
clean_2022_df.to_csv('cleaned_2022.csv', index=False)

In [18]:
#one more time for 2020
#load January 2020 data
df_2020 = pd.read_csv('JC-202001-citibike-tripdata.csv')

#rename to match other data and drop extraneous columns
df_2020 = df_2020.drop(['tripduration', 
                        'bikeid', 
                        'birth year', 
                        'gender', 
                        'start station id', 
                        'start station latitude', 
                        'start station longitude', 
                        'end station id',
                        'end station latitude',
                        'end station longitude'], axis=1)
df_2020 = df_2020.rename(columns={'starttime': 'started_at', 
                                  'stoptime': 'ended_at', 
                                  'start station name': 'start_station_name', 
                                  'end station name': 'end_station_name', 
                                  'usertype': 'member_casual'})

#merge with all_stations_df to add in corrected start lat and lng, then drop extra columns
almost_2020_df = pd.merge(df_2020, all_stations_df, left_on= 'start_station_name', right_on= 'station_name', how='left')
almost_2020_df = almost_2020_df.rename(columns={'latitude': 'start_lat', 'longitude': 'start_lng', 'station_id': 'start_station_id'}).drop(['station_name'], axis=1)

#merge with all_stations_df to add in corrected end lat and lng, then drop extra columns
clean_2020_df = pd.merge(almost_2020_df, all_stations_df, left_on= 'end_station_name', right_on= 'station_name', how='left')
clean_2020_df = clean_2020_df.rename(columns={'latitude': 'end_lat', 'longitude': 'end_lng', 'station_id': 'end_station_id'}).drop(['station_name'], axis=1)
clean_2020_df = clean_2020_df.replace('Subscriber', 'member')
clean_2020_df = clean_2020_df.replace('Customer', 'casual')

#export cleaned 2020 csv
clean_2020_df.to_csv('cleaned_2020.csv', index=False)


In [19]:
clean_2020_df.head()

,started_at,ended_at,start_station_name,end_station_name,member_casual,start_station_id,start_lat,start_lng,end_station_id,end_lat,end_lng
0,2020-01-01 00:04:50.1920,2020-01-01 00:08:37.0370,Grove St PATH,Newark Ave,member,JC115,40.719121,-74.043265,JC032,40.721525,-74.046305
1,2020-01-01 00:16:01.6700,2020-01-01 00:22:19.0800,Grove St PATH,Brunswick & 6th,member,JC115,40.719121,-74.043265,JC081,40.726180,-74.050369
2,2020-01-01 00:17:33.8770,2020-01-01 00:22:22.4420,Grove St PATH,Brunswick & 6th,casual,JC115,40.719121,-74.043265,JC081,40.726180,-74.050369
3,2020-01-01 00:32:05.9020,2020-01-01 00:39:21.0660,Sip Ave,Astor Place,casual,NaN,NaN,NaN,JC077,40.719179,-74.071339
4,2020-01-01 00:46:19.6780,2020-01-01 00:50:11.3440,Grove St PATH,Marin Light Rail,member,JC115,40.719121,-74.043265,JC013,40.714584,-74.042817
